In [ ]:
import json
import copy

from transformers import AutoTokenizer
from tokenizers.models import BPE

tokenizer = AutoTokenizer.from_pretrained("unsloth/Qwen2-0.5B-Instruct")
vocab = tokenizer.get_vocab()
tokenizer_json = json.loads(tokenizer._tokenizer.to_str())
merges = tokenizer_json["model"]["merges"]

aux_tokenizer = AutoTokenizer.from_pretrained("Rajeeb321/qwen-0.5_half_tokenizer_hi")
aux_vocab = aux_tokenizer.get_vocab()
aux_tokenizer_json = json.loads(aux_tokenizer._tokenizer.to_str())
aux_merges = aux_tokenizer_json["model"]["merges"]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# for merge in aux_merges:
#     token_1, token_2 = merge.split(" ")
#     token_1 = ''.join(tokenizer.tokenize(token_1))
#     token_2 = ''.join(tokenizer.tokenize(token_2))
#     a = token_1 + ' '+ token_2
#     print(' '.join(tokenizer.tokenize(merge)))
#     print(a)
#     # print(tokenizer.tokenize(merge) == a)

In [ ]:

# {k: v for k, v in sorted(vocab.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
[*aux_tokenizer_json["model"]]

['type',
 'dropout',
 'unk_token',
 'continuing_subword_prefix',
 'end_of_word_suffix',
 'fuse_unk',
 'byte_fallback',
 'ignore_merges',
 'vocab',
 'merges']

In [ ]:
[*tokenizer_json["model"]]

['type',
 'dropout',
 'unk_token',
 'continuing_subword_prefix',
 'end_of_word_suffix',
 'fuse_unk',
 'byte_fallback',
 'ignore_merges',
 'vocab',
 'merges']

In [ ]:
aux_tokenizer.vocab_size

10000

In [ ]:
# # merge the tokenizers
# num_new_token = 0
# max_new_token = 19000
# ret_vocab = copy.copy(vocab)
# ret_merges = []
# old_merges = copy.copy(merges)
# for merge in aux_merges:
#     # vocab
#     token_1, token_2 = merge.split(" ")
#     token = token_1 + token_2
#     if num_new_token < max_new_token:
#         if token_1 not in ret_vocab and token_2 not in ret_vocab: # both are new
#             ret_vocab[token_1] = len(vocab) + num_new_token
#             ret_vocab[token_2] = len(vocab) + num_new_token + 1
#             num_new_token += 2
#         elif token_1 not in ret_vocab and token_2 in ret_vocab: # new + existing
#             ret_vocab[token_1] = len(vocab) + num_new_token
#             num_new_token += 1
#         elif token_1 in ret_vocab and token_2 not in ret_vocab: # old + existing
#             ret_vocab[token_2] = len(vocab) + num_new_token
#             num_new_token += 1
#         else: # both are existing tokens
#             pass
#         if token not in ret_vocab:
#             ret_vocab[token] = len(vocab) + num_new_token
#             num_new_token += 1
#     # merge
#     if merge in merges:
#         old_merges.remove(merge)
#         ret_merges.append(merge)
#     elif token in ret_vocab and token_1 in ret_vocab and token_2 in ret_vocab:
#         ret_merges.append(merge)

In [ ]:
# # retrain tokenizer
# merges = old_merges + ret_merges
# vocab = ret_vocab
# tokenizer.backend_tokenizer.model = BPE(
#     vocab=vocab,
#     merges=[(merge.split(' ')[0], merge.split(' ')[1]) for merge in merges],
#     fuse_unk=False,
# )

In [ ]:
len(vocab)

151646

In [ ]:
num = 151646
ret_vocab = copy.copy(vocab)
for key, value in aux_vocab.items():
    if key not in vocab.keys():
      ret_vocab[key] = value + num

In [ ]:
len(aux_vocab)

10000

In [ ]:
len(ret_vocab)

158017

In [ ]:
# retrain tokenizer
merges =  aux_merges + merges
vocab = ret_vocab
tokenizer.backend_tokenizer.model = BPE(
    vocab=vocab,
    merges=[(merge.split(' ')[0], merge.split(' ')[1]) for merge in merges],
    fuse_unk=False,
)

In [ ]:
# save
tokenizer.save_pretrained("here")

('here/tokenizer_config.json',
 'here/special_tokens_map.json',
 'here/vocab.json',
 'here/merges.txt',
 'here/added_tokens.json',
 'here/tokenizer.json')

In [ ]:
tokenizer_1 = AutoTokenizer.from_pretrained("unsloth/Qwen2-0.5B-Instruct")
tokenizer_2 = AutoTokenizer.from_pretrained("here")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenizer_1.vocab_size, tokenizer_2.vocab_size, tokenizer_2.vocab_size - tokenizer_1.vocab_size

(151643, 158017, 6374)

In [ ]:
text = """प्रकाश के लिए उपयोग किए जाने वाले योजक रंग प्रणाली में, प्राथमिक रंग लाल, हरे और नीले (आरजीबी) हैं। प्रकाश के लिए उपयोग किए जाने वाले योजक रंग प्रणाली में, प्राथमिक रंग लाल, हरे और नीले (आरजीबी) हैं।"""
print(len(tokenizer_1.tokenize(text)))
# 81

# tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-3-8b-Instruct")
print(len(tokenizer_2.tokenize(text)))
print(len(aux_tokenizer.tokenize(text)))
# # 46

178
102
98


In [ ]:

text = "A neural network is a method in artificial intelligence that teaches computers to process data in a way that is inspired by the human brain. It is a type of machine learning process, called deep learning, that uses interconnected nodes or neurons in a layered structure that resembles the human brain."
text ="""The human brain is the inspiration behind neural network architecture. Human brain cells, called neurons, form a complex, highly interconnected network and send electrical signals to each other to help humans process information. Similarly, an artificial neural network is made of artificial neurons that work together to solve a problem. Artificial neurons are software modules, called nodes, and artificial neural networks are software programs or algorithms that, at their core, use computing systems to solve mathematical calculations.

Simple neural network architecture
A basic neural network has interconnected artificial neurons in three layers:

Input Layer
Information from the outside world enters the artificial neural network from the input layer. Input nodes process the data, analyze or categorize it, and pass it on to the next layer.

Hidden Layer
Hidden layers take their input from the input layer or other hidden layers. Artificial neural networks can have a large number of hidden layers. Each hidden layer analyzes the output from the previous layer, processes it further, and passes it on to the next layer.

Output Layer
The output layer gives the final result of all the data processing by the artificial neural network. It can have single or multiple nodes. For instance, if we have a binary (yes/no) classification problem, the output layer will have one output node, which will give the result as 1 or 0. However, if we have a multi-class classification problem, the output layer might consist of more than one output node.

Deep neural network architecture
Deep neural networks, or deep learning networks, have several hidden layers with millions of artificial neurons linked together. A number, called weight, represents the connections between one node and another. The weight is a positive number if one node excites another, or negative if one node suppresses the other. Nodes with higher weight values have more influence on the other nodes.
Theoretically, deep neural networks can map any input type to any output type. However, they also need much more training as compared to other machine learning methods. They need millions of examples of training data rather than perhaps the hundreds or thousands that a simpler network might need."""
print(len(tokenizer_1.tokenize(text)))
print(len(tokenizer_2.tokenize(text)))

print(tokenizer_1.tokenize(text))
print(tokenizer_2.tokenize(text))

418
419
['The', 'Ġhuman', 'Ġbrain', 'Ġis', 'Ġthe', 'Ġinspiration', 'Ġbehind', 'Ġneural', 'Ġnetwork', 'Ġarchitecture', '.', 'ĠHuman', 'Ġbrain', 'Ġcells', ',', 'Ġcalled', 'Ġneurons', ',', 'Ġform', 'Ġa', 'Ġcomplex', ',', 'Ġhighly', 'Ġinterconnected', 'Ġnetwork', 'Ġand', 'Ġsend', 'Ġelectrical', 'Ġsignals', 'Ġto', 'Ġeach', 'Ġother', 'Ġto', 'Ġhelp', 'Ġhumans', 'Ġprocess', 'Ġinformation', '.', 'ĠSimilarly', ',', 'Ġan', 'Ġartificial', 'Ġneural', 'Ġnetwork', 'Ġis', 'Ġmade', 'Ġof', 'Ġartificial', 'Ġneurons', 'Ġthat', 'Ġwork', 'Ġtogether', 'Ġto', 'Ġsolve', 'Ġa', 'Ġproblem', '.', 'ĠArtificial', 'Ġneurons', 'Ġare', 'Ġsoftware', 'Ġmodules', ',', 'Ġcalled', 'Ġnodes', ',', 'Ġand', 'Ġartificial', 'Ġneural', 'Ġnetworks', 'Ġare', 'Ġsoftware', 'Ġprograms', 'Ġor', 'Ġalgorithms', 'Ġthat', ',', 'Ġat', 'Ġtheir', 'Ġcore', ',', 'Ġuse', 'Ġcomputing', 'Ġsystems', 'Ġto', 'Ġsolve', 'Ġmathematical', 'Ġcalculations', '.ĊĊ', 'Simple', 'Ġneural', 'Ġnetwork', 'Ġarchitecture', 'Ċ', 'A', 'Ġbasic', 'Ġneural', 'Ġnetwork', '

In [ ]:
tokenizer_2.push_to_hub("universalml0/phi3.5-merged-tokenizer-original_with_Nepali", token="hf_" )

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/universalml0/phi3.5-merged-tokenizer-original_with_Nepali/commit/3d20d2a2f98d301d19fc75760d2c492ed0363288', commit_message='Upload tokenizer', commit_description='', oid='3d20d2a2f98d301d19fc75760d2c492ed0363288', pr_url=None, pr_revision=None, pr_num=None)